In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
EPOCHS = 1
WINDOW_SIZE = 360

In [46]:
def generate_data(filename):
    df = pd.read_csv(filename)
    if filename == "training_by_classes.csv" or filename == "validating_by_classes.csv":
        df = df[df.iloc[:,1]=='N']
#         print(df.head())
#     print(df.shape)
    for _ in range(EPOCHS):
        for row in range(len(df)):
    #         print(row)
            X = df.iloc[row, WINDOW_SIZE-int(WINDOW_SIZE/2):WINDOW_SIZE+int(WINDOW_SIZE/2)]
            X = X.to_numpy().reshape(1, 360, 1).astype('float32')
    #         print(X)
            if filename == "testing_by_classes.csv":
                yield(X, df.iloc[row, 1])
            else:
                yield(X, X)

In [36]:
# encoder_input = keras.Input(shape=(360, 1), name='input')
# x = keras.layers.Dense(360, activation='relu')(encoder_input)
# x = keras.layers.LSTM(180, return_sequences=False)(x)
# encoder_output = keras.layers.RepeatVector(WINDOW_SIZE)(x)
# encoder = keras.Model(encoder_input, encoder_output, name='encoder')
# encoder.summary()

# decoder_input = keras.Input(shape=(360, 180), name='decoder_input')
# x = keras.layers.LSTM(180, return_sequences=True)(encoder_output)
# x = keras.layers.Dense(360, activation='relu')(decoder_input)
# decoder_output = keras.layers.TimeDistributed(keras.layers.Dense(1), name='output')(x)
# decoder = keras.Model(decoder_input, decoder_output, name='decoder')
# decoder.summary()

# autoencoder = keras.Model(encoder_input, decoder(encoder(encoder_input)), name='autoencoder')
# autoencoder.compile(loss='mse', optimizer='adam')
# autoencoder.summary()

encoder_input = keras.Input(shape=(WINDOW_SIZE, 1), name='input')
# x = keras.layers.Flatten()(encoder_input)
x = keras.layers.Dense(360, activation='tanh')(encoder_input)
x = keras.layers.LSTM(60, return_sequences=False)(x)
encoder_output = keras.layers.Reshape((60,1))(x)
# encoder_output = keras.layers.LSTM(45, return_sequences=False)(x)

encoder = keras.Model(encoder_input, encoder_output, name='encoder')
encoder.summary()

decoder_input = keras.layers.Input(shape=(60, 1), name='decoder_input')
x = keras.layers.LSTM(60, return_sequences=False)(decoder_input)
x = keras.layers.Dense(360)(x)
decoder_output = keras.layers.Reshape((360, 1))(x)
# x = keras.layers.LSTM(90, return_sequences=True)(x)
# decoder_output = keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(x)

decoder = keras.Model(decoder_input, decoder_output, name='decoder')
decoder.summary()

opt = keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

autoencoder = keras.Model(encoder_input, decoder(encoder(encoder_input)), name='autoencoder')
autoencoder.summary()
autoencoder.compile(loss='mse', optimizer=opt)

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 360, 1)]          0         
_________________________________________________________________
dense_15 (Dense)             (None, 360, 360)          720       
_________________________________________________________________
lstm_48 (LSTM)               (None, 60)                101040    
_________________________________________________________________
reshape_21 (Reshape)         (None, 60, 1)             0         
Total params: 101,760
Trainable params: 101,760
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 60, 1)]           0         
____________________________________

In [37]:
autoencoder.summary()
autoencoder.fit(generate_data("training_by_classes.csv"), steps_per_epoch=42020, epochs=EPOCHS, shuffle=True)
# for X, _ in generate_data("training_by_classes.csv"):
#     print(X.shape)
#     autoencoder.fit(X, X)
#     encoded = encoder.predict(X)
#     print(encoded.shape)
#     decoded = decoder.predict(encoded)
#     print(decoded.shape)
#     plt.plot(X.flatten(), label='Original')
#     plt.plot(encoded[0,:,0], label='Encoded')
#     plt.plot(decoded.flatten(), label='Decoded')
#     plt.legend()
#     plt.show()
    

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 360, 1)]          0         
_________________________________________________________________
encoder (Functional)         (None, 60, 1)             101760    
_________________________________________________________________
decoder (Functional)         (None, 360, 1)            36840     
Total params: 138,600
Trainable params: 138,600
Non-trainable params: 0
_________________________________________________________________
(42020, 722)
42020/42020 [==============================] - 429s 10ms/step - loss: 0.0494


In [58]:
abnormal_errors = np.array([])
normal_errors = np.array([])
for X, y in generate_data("testing_by_classes.csv"):
#     print("X", X.shape)
    encoded = encoder.predict(X)
#         print("Encoder", encoded.shape)
    decoded = decoder.predict(encoded)
#         print("Decoder", decoded.shape)
    error = keras.losses.mae(X.flatten(), decoded.flatten()).numpy()
    if y != 'N':
        if errors.shape[0] <= 0:
            abnormal_errors = np.array([error]).reshape(-1, 1)
        else:
            abnormal_errors = np.append(errors, np.array(error).reshape(-1, 1), axis=1)
    else:
        if errors.shape[0] <= 0:
            normal_errors = np.array([error]).reshape(-1, 1)
        else:
            normal_errors = np.append(errors, np.array(error).reshape(-1, 1), axis=1)
print(np.mean(abnormal_errors))
print(np.mean(normal_errors))
#     plt.plot(X.flatten(), label='Original')
#     plt.plot(encoded.flatten(), label='Encoded')
#     plt.plot(decoded.flatten(), label='Decoded')
#     plt.title(y+": "+str(error))
#     plt.legend()
#     plt.show()

(1, 81)
(26996, 722)
0.26036087
0.25686723


In [59]:
print(np.std(abnormal_errors))
print(np.std(normal_errors))

0.1676401
0.16765527
